**Input**

In [ ]:
day = 19

import pyodide
def read(name):
  return pyodide.open_url('https://raw.githubusercontent.com/joaomoreno/aoc2021/main/%d/%s' % (day, name)).getvalue()

sample = read('sample')
input = read('input')

**Parsing**

In [ ]:
import numpy as np
import re
rx = re.compile(r'^(-?\d+),(-?\d+),(-?\d+)$')

def parse(input):
  result = []
  scanner = []

  for line in input.splitlines(False):
    if line == '':
      result.append(np.array(scanner, np.int32))
      scanner = []
    elif match := rx.match(line):
      scanner.append(list(map(int, match.groups())))
  
  result.append(np.array(scanner, np.int32))
  return result

**Part One**

In [8]:
from itertools import chain

def rotx(a): return np.array([[1, 0, 0], [0, np.cos(a), -np.sin(a)], [0, np.sin(a), np.cos(a)]], np.int32)
def roty(a): return np.array([[np.cos(a), 0, np.sin(a)], [0, 1, 0], [-np.sin(a), 0, np.cos(a)]], np.int32)
def rotz(a): return np.array([[np.cos(a), -np.sin(a), 0], [np.sin(a), np.cos(a), 0], [0, 0, 1]], np.int32)
def radian(step): return step * np.pi / 2

def spin(rot):
  for i in range(4):
    yield rot(radian(i))

def all_rotations():
  for a in chain(spin(roty), (rotx(radian(1)), rotx(radian(3)))):
    for b in spin(rotz):
      yield a @ b

rotations = list(all_rotations())

def find_transform(known, points):
  for r in rotations:
    rotated_points = points @ r.T
    for p in rotated_points[12:]:
      for root in known:
        v = p - root
        count = 0

        for q in rotated_points:
          if tuple(q - v) in known:
            count += 1
          if count >= 12:
            return (r, v)

def run(input):
  beacons = set(tuple(p) for p in input[0])
  todo = list(range(1, len(input)))
  
  while len(todo) > 0:
    for i in range(len(todo)):
      points = input[todo[i]]
      if transform := find_transform(beacons, points):
        r, v = transform
        beacons |= set(tuple(p - v) for p in points @ r.T)
        todo.pop(i)
        break
  
  return len(beacons)

run(parse(sample))

79

**Part Two**

In [9]:
from itertools import combinations

def dist(a, b):
  return np.sum(np.abs(a - b))

def run(input):
  beacons = set(tuple(p) for p in input[0])
  todo = list(range(1, len(input)))
  scanners = []
  
  while len(todo) > 0:
    for i in range(len(todo)):
      points = input[todo[i]]
      if transform := find_transform(beacons, points):
        r, v = transform
        beacons |= set(tuple(p - v) for p in points @ r.T)
        scanners.append(-v)
        todo.pop(i)
        break
  
  return max(dist(a, b) for a, b in combinations(scanners, 2))

run(parse(sample))

3621